In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('/Users/Seb/development/redi/data-analysis-project/data/fatalencounters.csv')

In [5]:
smaller_df = df.drop([
    'Timestamp', 'URL of image of deceased',
    '20791', 'Date&Description',
    'State Data Status Prior to Jan. 1, 2013. All states complete after Jan. 1, 2013',
    'Link to news article or photo of official document',
    'Official disposition of death (justified or other)',
    'A brief description of the circumstances surrounding the death',
    'Unique identifier/submitted by'], axis=1)

## Trying to plot locations

In [6]:
location_cols = [
    'Location of injury (address)',
    'Location of death (city)', 'Location of death (state)',
    'Location of death (zip code)','Location of death (county)'
]

In [7]:
clean_locations_df = smaller_df.dropna(axis=0, subset=location_cols)

In [8]:
test_row = clean_locations_df.iloc[0].copy()

In [9]:
location_keys = [column_name for column_name in test_row.index.tolist() if column_name.startswith('Location')]

In [10]:
location_colnames = [column_name for column_name in test_row.index.tolist()
                     if column_name.startswith('Location')]

def combine_location_details(row, location_keys=location_colnames):
    return ', '.join(row[location_keys].values.tolist())
    

In [88]:
%%time
addresses = []
for row, values in clean_locations_df.head(100).iterrows():
    try:
        addresses.append(combine_location_details(values))
    except TypeError as err:
        print(err)
        print(row)
        print(values[location_colnames])

CPU times: user 81.1 ms, sys: 2.58 ms, total: 83.7 ms
Wall time: 88.3 ms


In [89]:
coords = [geocoder.google(address).latlng for address in addresses]

In [90]:
import folium
import geocoder 
import numpy as np

In [92]:
for coord in coords:
    if len(coord) > 0:
        y = coord[1]
        x = coord[0]
        folium.CircleMarker([x, y], color='black', fill_color='#2EFE2E', radius=5, fill_opacity=1).add_to(map_1)

In [94]:
map_1.save('test.html')